## import libraries

### pip install

In [1]:
#pip install
!pip install faiss-gpu --quiet
!pip install boto3 --quiet
!pip install langchain-aws --quiet
!pip install langchain-community boto3 --quiet
!pip install boto3 requests requests-aws4auth --quiet
!pip install pymongo --quiet
!pip install google-search-results sagemaker --quiet
!pip install boto3 nltk --quiet
!pip install wikipedia --quiet
!pip install tavily-python --quiet
!pip install --upgrade langchain sympy numexpr --quiet
!pip install langchain-experimental --quiet
!pip install -U langgraph langchain_community langchain_anthropic langchain_experimental --quiet
!pip install langchain-redis langchain-openai redis --quiet
#!docker run -d -p 6379:6379 redis:latest --quiet

### libraries

In [2]:
import boto3
import os
from langchain import hub

#chat models
from langchain_aws import ChatBedrock
from langchain_aws import ChatBedrockConverse

#LLMs
from langchain_aws import BedrockLLM
from langchain_community.llms import AmazonAPIGateway
from langchain_aws import SagemakerEndpoint

#prompts
from langchain.prompts import PromptTemplate

#embedding models
from langchain_aws import BedrockEmbeddings
# from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import ContentHandlerBase

#document loaders
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
from langchain_community.document_loaders import AmazonTextractPDFLoader

#vector stores
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.vectorstores import DocumentDBVectorSearch

from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

#retrievers
from langchain_aws import AmazonKendraRetriever
from langchain_aws import AmazonKnowledgeBasesRetriever

#memory
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables import RunnablePassthrough

#graphs
from langchain_community.graphs import NeptuneGraph
from langchain_community.graphs import NeptuneAnalyticsGraph
from langchain_community.chains.graph_qa.neptune_cypher import NeptuneOpenCypherQAChain

from langchain_community.graphs import NeptuneRdfGraph
from langchain_community.chains.graph_qa.neptune_sparql import NeptuneSparqlQAChain

#callbacks
from langchain_community.callbacks.bedrock_anthropic_callback import BedrockAnthropicTokenUsageCallbackHandler
from langchain_community.callbacks import SageMakerCallbackHandler

#chains
# from langchain_experimental.comprehend_moderation import AmazonComprehendModerationChain

#tavily
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults

#tools
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.tools import BaseTool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, CSVLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

#agent
from langchain.agents import create_tool_calling_agent, AgentType
from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI

USER_AGENT environment variable not set, consider setting it to identify your requests.


## setup agent

### setup llm

In [3]:
model_parameter = {"temperature": 0.9, "top_p": .5, "max_tokens_to_sample": 200}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3.client('bedrock'),
    model_kwargs=model_parameter, 
    beta_use_converse_api=True
)

In [4]:
embeddings = BedrockEmbeddings(
    client=boto3.client('bedrock-runtime', region_name="us-east-1"),
    model_id = "amazon.titan-embed-text-v2:0"
)

### initiate tools

In [5]:
os.environ["TAVILY_API_KEY"] = "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"
search = TavilySearchResults()

In [44]:
prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],  # Include both input and agent_scratchpad
    template="""
    
If the input is to do with retail or an item to buy, do the following:

Use the retriever tool to answer any queries related to internal data files. 
Only use external tools (like Wikipedia) if no relevant information is found internally.
When asked about a specific item output the answers to the following queries: 

Should this item be ordered? 
Find out what date it is currently using date time tool. Convert the month and year into strings and use these for the rest of the answer.
Use recent stock statistics to see if the stocks for this month are low compared to how much was used last month.
Use the statistics from last year to predict how much of each item will be needed for the rest of this month and next month.
Use what month it is at the time the call is being made and whether any important holidays eg christmas or summer will be coming up and use this information in your decision.
a) how much should be ordered for this month and next month
b) how much should be ordered in the next few months


What brand?
Use the customers reviews to decide this. 

If this item should be ordered, how much of this item should be ordered?
Use the repl tool to calculate these numbers.

In your final output:
1. Explain your reasoning for whether an item should be ordered and how much.
Don't explain your reasoning for
Explain the maths you used.
2. Draft an email to an imaginary supplier of this brand to order the number of the item you think should be ordered, signing off as Reply Auto Replenishment.

Else:
Answer the question appropriately using multiple tools to verify your answer

Query: {input}

{agent_scratchpad}
"""
)


In [45]:
# Define the path to the directory containing CSV files
data_files_directory = "data_files"

# Use DirectoryLoader to load all CSV files
loader = DirectoryLoader(
    data_files_directory,
    glob="**/*.csv",  # Match all CSV files in the directory (including subdirectories)
    loader_cls=CSVLoader  # Use CSVLoader for parsing the files
)

In [46]:
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "Retriever_tool",
    "First search internal data files to answer any queries. If there is no useful information then use other tools.",
)

In [47]:
#datetime tool
from datetime import datetime
datetime_tool = Tool(
    name="Datetime",
    func=lambda x: datetime.now().isoformat(),
    description="Returns the current datetime",
)


In [48]:
#loading in tools

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Use this to execute python commands and calculations.",
    func=python_repl.run,
)
wikipedia_tool = load_tools(["wikipedia"], llm=bedrock_llm)

tools = [datetime_tool,retriever_tool,search,repl_tool,wikipedia_tool[0]]

# running agent

In [49]:
# def run_agent(prompt,agent_input,raw_list,verbose_setting):
#     agent = create_tool_calling_agent(bedrock_llm, tools, prompt)
#     agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose_setting,handle_parsing_errors=True)
#     response = agent_executor.invoke({"input": agent_input})
#     raw_output = response['output'][0]['text']
#     raw_list.append(raw_output)

In [50]:
agent = create_tool_calling_agent(bedrock_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False,handle_parsing_errors=True)

In [51]:
# raw_list = []
# run_agent(prompt,'christmas pudding',raw_list,False)
# run_agent(prompt,'suncream',raw_list,False)
# for i in range(len(raw_list)):
#     print('-'*20)
#     print(raw_list[i])
#     print('\n') 

In [52]:
!pip install gradio boto3

In [53]:
# !pip install gradio==3.16.1b1

In [55]:
import gradio as gr
from gradio import ChatMessage

# Mock interaction function for Gradio
async def interact_with_langchain_agent(prompt, chat_history):
    """
    Handles interaction between the user and the LangChain agent.
    """
    # Append the user's message to the chat history
    chat_history.append({"role": "user", "content": prompt})
    yield chat_history

    # Process the input with the LangChain agent
    try:
        response = agent_executor.invoke({"input": prompt})  # Replace with your agent logic
        raw_output = response["output"][0]["text"]  # Adjust indexing if necessary
        chat_history.append({"role": "assistant", "content": raw_output})
    except Exception as e:
        # Handle parsing errors or any other issues
        chat_history.append({"role": "assistant", "content": f"Error: {str(e)}"})

    # Yield the updated chat history
    yield chat_history


with gr.Blocks() as demo:
    gr.Markdown("# Auto Replenishment Chatbot 🦜⛓️")
    
    # Chatbot UI
    chatbot = gr.Chatbot(label="Agent")
    user_input = gr.Textbox(lines=1, placeholder="Enter your query here...", label="Your Query")

    # Link input to the interaction function
    user_input.submit(
        interact_with_langchain_agent,
        inputs=[user_input, chatbot],
        outputs=chatbot
    )

# Launch the Gradio interface
demo.launch(share=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
which: no node in (/usr/local/cuda-11.8/bin:/home/ec2-user/anaconda3/envs/python3/bin:/home/ec2-user/anaconda3/condabin:/home/ec2-user/anaconda3/bin:/opt/amazon/openmpi/bin:/opt/amazon/efa/bin:/home/ec2-user/.dl_binaries/bin:/usr/local/cuda-12.1/bin:/usr/local/cuda-12.1/include:/usr/libexec/gcc/x86_64-redhat-linux/7:/opt/aws/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin)


* Running on local URL:  http://127.0.0.1:7884
* Running on public URL: https://91b476af595a14391d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [54]:
from gradio import ChatMessage

async def interact_with_langchain_agent(prompt, messages):
    messages.append(ChatMessage(role="user", content=prompt))
    yield messages
    async for chunk in agent_executor.astream(
        {"input": prompt}
    ):
        if "steps" in chunk:
            for step in chunk["steps"]:
                messages.append(ChatMessage(role="assistant", content=step.action.log,
                                  metadata={"title": f"🛠️ Used tool {step.action.tool}"}))
                yield messages
        if "output" in chunk:
            messages.append(ChatMessage(role="assistant", content=chunk["output"]))
            yield messages


with gr.Blocks() as demo:
    gr.Markdown("Auto replenishment Reply")
    chatbot_2 = gr.Chatbot(
        type="messages",
        label="Agent",
        avatar_images=(
            None,
            "https://em-content.zobj.net/source/twitter/141/parrot_1f99c.png",
        ),
    )
    input_2 = gr.Textbox(lines=1, label="Chat Message")
    input_2.submit(interact_with_langchain_agent, [input_2, chatbot_2], [chatbot_2])
    
demo.launch(share=True)

which: no node in (/usr/local/cuda-11.8/bin:/home/ec2-user/anaconda3/envs/python3/bin:/home/ec2-user/anaconda3/condabin:/home/ec2-user/anaconda3/bin:/opt/amazon/openmpi/bin:/opt/amazon/efa/bin:/home/ec2-user/.dl_binaries/bin:/usr/local/cuda-12.1/bin:/usr/local/cuda-12.1/include:/usr/libexec/gcc/x86_64-redhat-linux/7:/opt/aws/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin)


* Running on local URL:  http://127.0.0.1:7881
* Running on public URL: https://556bf6da615f54c901.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [19]:
import gradio as gr
from botocore.exceptions import ClientError

bedrock_client = boto3.client("bedrock", region_name="us-east-1")

history = ["", ""]
                    
    
    
# response = agent_executor.invoke("input":"hi")
# print(f"Agent response: {response}")
    
    
def chat_with_bedrock(message,history):
    try:
        # Pass the message to the agent and get the response
        response = agent_executor.invoke({"input":message})
        print(f"Agent response: {response}")
        history.append([message, response])
        
        return [message,response], history
    
    except Exception as e:
        print(f"Error calling Bedrock model: {str(e)}")
        return f"Error calling Bedrock model: {str(e)}"

    

# Create a Gradio chat interface
def gradio_interface():
    return gr.ChatInterface(fn=chat_with_bedrock, type="messages")

# Launch the Gradio interface
gradio_interface().launch(share=True)

which: no node in (/usr/local/cuda-11.8/bin:/home/ec2-user/anaconda3/envs/python3/bin:/home/ec2-user/anaconda3/condabin:/home/ec2-user/anaconda3/bin:/opt/amazon/openmpi/bin:/opt/amazon/efa/bin:/home/ec2-user/.dl_binaries/bin:/usr/local/cuda-12.1/bin:/usr/local/cuda-12.1/include:/usr/libexec/gcc/x86_64-redhat-linux/7:/opt/aws/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://64ed0f2e284c10ed34.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Agent response: {'input': 'hi', 'output': [{'type': 'text', 'text': 'Since the query "hi" is a simple greeting, here is a friendly response:\n\nHi there! How can I assist you today? I\'m an AI assistant created by Anthropic to help with a variety of tasks. Please let me know if you have any questions or if there is anything specific I can help with.\n\nI searched Wikipedia for some context on the word "hi", but it seems to mostly refer to a song by Wings and a British sitcom called "Hi-de-Hi!". Neither of those seem directly relevant to your greeting.\n\nIn general, "hi" is an informal way of saying hello and greeting someone. It\'s a common and friendly way to start a conversation or interaction. Let me know if you need any other information or if you have a particular topic you\'d like to discuss further.', 'index': 0}]}


Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/blocks.py", line 2053, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/blocks.py", line 1859, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/gradio/components/chatbot.py", line 553, in postprocess
    self._check_format(value, "messages")
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3